In [ ]:
import numpy as np
from modules.preprocessing.file_creation import create_mat_files
from modules.feature_extraction.feature_extraction import get_psd_feature_vectors
from modules.training.training import train_clf_and_get_metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from modules.evaluation.evaluation import evaluate_clf_and_get_metrics

: 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd '/content/drive/MyDrive/Doctorado/Doctorados/Repositorio'

### Orden en el que se leyeron los archivos:
Vertiz - Acevedo - AguilarGonzales - Carrere - Chaves - Poux - Quiroga - Tabernig

### Resultados con LDA:
61.67, (70)41.67, 63.33, 53.33, 41.67, (53,3)48.33, 73.33, 55.0

## Importo datos para calibración

In [ ]:
dict_cal = create_mat_files('./data/im_tention_signals', file_type='calibration')
dict_ter = create_mat_files('./data/im_tention_signals', file_type='therapy', ther_number_of_trials=60)

print(dict_cal['subject_1']['mi_rest'].shape)
print(dict_ter['subject_1']['mi'].shape)
print(dict_ter['subject_1']['target'].shape)

Number of subjects found: 8
Number of MI calibration trials for subject1: 30
Number of MI calibration trials for subject2: 30
Number of MI calibration trials for subject3: 30
Number of MI calibration trials for subject4: 30
Number of MI calibration trials for subject5: 30
Number of MI calibration trials for subject6: 30
Number of MI calibration trials for subject7: 30
Number of MI calibration trials for subject8: 30
Number of subjects found: 8
Number of therapy trials for subject1: 60
Number of therapy trials for subject2: 60
Number of therapy trials for subject3: 60
Number of therapy trials for subject4: 60
Number of therapy trials for subject5: 60
Number of therapy trials for subject6: 60
Number of therapy trials for subject7: 60
Number of therapy trials for subject8: 60
(500, 5, 60)
(500, 5, 60)
(60,)


In [ ]:
dict_cal['subject_1']['mi_rest'].shape
dict_ter['subject_1']['mi'].shape[2]

60

In [ ]:
y_calibration = np.hstack((np.ones(30, dtype=np.int8), np.zeros(30, dtype=np.int8)))

In [ ]:
psd_matrix = get_psd_feature_vectors(dict_cal['subject_1']['mi_rest'])
psd_matrix.shape

(60, 23)

## Entrenamiento usando LDA con datos de calibración

In [ ]:
clasificadores_por_sujeto_lda = []
acc_train_lda = []
tpr_train_lda = []
for i in range(8):
    psd_matrix = get_psd_feature_vectors(dict_cal[f'subject_{i+1}']['mi_rest'])
    lda, metrics =train_clf_and_get_metrics(psd_matrix, y_calibration, clf=LinearDiscriminantAnalysis())
    clasificadores_por_sujeto_lda.append(lda)
    acc_train_lda.append(metrics.acc)
    tpr_train_lda.append(metrics.tpr)

acc_train_lda

[83.33, 95.0, 98.33, 85.0, 90.0, 93.33, 80.0, 96.67]

In [ ]:
tpr_train_lda

[86.67, 93.33, 96.67, 80.0, 93.33, 93.33, 86.67, 100.0]

## LDA entrenados con datos de terapia

In [ ]:
tpr_terapia_lda = []
acc_terapia_lda_IM = []
for i in range(8):
    psd_matrix = get_psd_feature_vectors(dict_ter[f'subject_{i+1}']['mi'])
    clf = clasificadores_por_sujeto_lda[i]
    # print(psd_matrix.shape)
    n_datos = psd_matrix.shape[0]
    y_therapy = np.array([i%2 for i in range(1, n_datos + 1)])
    # print(y_therapy)
    metrics = evaluate_clf_and_get_metrics(psd_matrix, clf, y_therapy)
    acc_terapia_lda_IM.append(metrics.acc)
    tpr_terapia_lda.append(metrics.tpr)

In [ ]:
acc_terapia_lda_IM

[41.67, 53.33, 41.67, 48.33, 55.0, 61.67, 63.33, 73.33]

In [ ]:
tpr_terapia_lda

[30.0, 50.0, 60.0, 56.67, 60.0, 66.67, 73.33, 76.67]